<a href="https://colab.research.google.com/github/Salvoaf/BreastCancerClassification/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from google.colab import drive
from google.colab.patches import cv2_imshow
import os
import cv2
import random
import shutil
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm.auto import tqdm
import csv

In [2]:
import os
from google.colab import files
files.upload()
!pip install kaggle
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/breast-histopathology-images
!unzip breast-histopathology-images.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating: IDC_regular_p

In [7]:
from os import listdir

base_path = "../content/IDC_regular_ps50_idx5/"
folder = listdir(base_path)
len(folder)

279

In [17]:
healthy_patch =  "../content/healthy_patch"
cancer_patch =  "../content/cancer_patch"
!mkdir healthy_patch
!mkdir cancer_patch

mkdir: cannot create directory ‘healthy_patch’: File exists
mkdir: cannot create directory ‘cancer_patch’: File exists


In [ ]:
import shutil

first = True
check_dim = True
for patches_each_patient in folder:
  path_patient = base_path +"/"+ patches_each_patient  #"../content/IDC_regular_ps50_idx5/id_patient"
  classes = listdir(path_patient)
  for c in classes:
    path_patient_class = path_patient +"/"+ c   #"../content/IDC_regular_ps50_idx5/id_patient/class0-1"
    for patch in listdir(path_patient_class):
      path_patient_patch = path_patient_class + "/" + patch
      if c == "0":
        shutil.copy(path_patient_patch, healthy_patch)
      elif c == "1":
        shutil.copy(path_patient_patch, cancer_patch)
      if first == False:
        if img.shape != mpimg.imread(path_patient_patch).shape:
          check_dim = False
total_cancer_patches = listdir(cancer_patch)
total_healthy_patches = listdir(healthy_patch)
total_patches = total_cancer_patches + total_healthy_patches

print("Number of cancer patches: " + str(len(total_cancer_patches)))
print("Number of healthy patches: " + str(len(total_healthy_patches))) 
print("Totla number patches: "+ str(len(total_patches)))

print("All image have same size: "+ str(check_dim))

In [32]:
print("Number of cancer patches: " + str(len(total_cancer_patches)))
print("Number of healthy patches: " + str(len(total_healthy_patches))) 
print("Totla number patches: "+ str(len(total_patches)))

Number of cancer patches: 78786
Number of healthy patches: 198739
Totla number patches: 277525


#Split the dataset

In [33]:
def split_dataset(dataset, dataset_size, train_percentage=0.6, val_percentage=0.2, test_percentage=0.2, shuffle=True):
  """
    split_dataset splits the dataset into training, validation and test sets.

    :param dataset: a list representing the whole dataset
    :param dataset_size: number of elements in the dataset
    :param train_percentage: the percentage of the dataset that will be used for training
    :param val_percentage: the percentage of the dataset that will be used for validation
    :param test_percentage: the percentage of the dataset that will be used for testing
    :param shuffle: if True the elements of the dataset will be randomly shuffled
    :return: three lists representing the training, validation and test sets
  """
  if train_percentage + val_percentage + test_percentage != 1:
    print('Total of percentages must be 1')
    return None, None, None
  
  if shuffle:
    random.shuffle(dataset)
    
  train_size = int(train_percentage * dataset_size)
  val_size = int(val_percentage * dataset_size)

  train_set = dataset[0:train_size]
  val_set = dataset[train_size:train_size+val_size]
  test_set = dataset[train_size+val_size:dataset_size]
    
  return train_set, val_set, test_set

In [36]:
random.seed(24) # seed is specified to always obtain the same results
training_set, validation_set, test_set = split_dataset(total_patches, len(total_patches))

num_training_samples = len(training_set)
num_validation_samples = len(validation_set)
num_test_samples = len(test_set)

print("Number of samples in the training set: " + str(num_training_samples))
print("Number of samples in the validation set: " + str(num_validation_samples))
print("Number of samples in the test set: " + str(num_test_samples))

Number of samples in the training set: 166515
Number of samples in the validation set: 55505
Number of samples in the test set: 55505


# Solving the Unbalanced Classes Issue
As we can see from the histogram below, the number of images for the two classes is not the same. 

In [ ]:
all_samples = [entry for entry in training_set if entry.split(".")[0][-3:] == 'all']
hem_samples = [entry for entry in training_set if entry.split(".")[0][-3:] == 'hem']

labels = [entry.split(".")[0][-3:] for entry in training_set]
_ = plt.hist(labels, bins=2)

print("Number of cancer cell images: " + str(len(all_samples)))
print("Number of normal cell images: " + str(len(hem_samples)))